In [4]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
import datetime 
from datetime import timedelta
import pytz
from pytz import timezone
from numpy import load
import os
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA as sklearnPCA
import h5py
import pylab
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import librosa
import matplotlib.dates as md
import sys

In [8]:
sys.path.insert(1, '../modules')
import data
import display

In [3]:
h5 = h5py.File('../data/sound_data_improved.hdf5', 'r')

In [14]:
d = h5['sound_data']

In [5]:
df = pd.read_csv('../output/june_2019_df_test.csv', low_memory = False)

In [6]:
df.head()

,Unnamed: 0,dBAS,assignment,median_dBAS
0,2019-06-01 00:00:42-04:00,63.53,2.0,61.01
1,2019-06-01 00:00:43-04:00,63.73,1.0,63.16
2,2019-06-01 00:00:44-04:00,64.94,2.0,64.48
3,2019-06-01 00:00:45-04:00,63.09,2.0,63.70
4,2019-06-01 00:00:46-04:00,61.66,2.0,63.76


# Creating subsample of 10000 points from all four sensors

In [34]:
sample_nums = np.random.choice(range(d.shape[0]), 10000, replace = False)

In [35]:
index = np.zeros(d.shape[0]).astype('bool')
index[sample_nums] = True

In [9]:
sensor_name = b'sonycnode-b827ebc178d2.sonyc'

In [13]:
h5.keys()

<KeysViewHDF5 ['sound_data']>

In [22]:
sensor_mask = data.get_sensor_mask(sensor_name, d['sensor_id'])

In [27]:
sensor_file_names.shape

(1423048,)

In [24]:
sensor_file_names = d['file_path'][sensor_mask]

In [28]:
sensor_time_stamp = d['file_path'][sensor_mask]

Changing feature vectors to assignments

In [31]:
pca_45 = sklearnPCA(45)
projected_45 = pca_45.fit_transform(d['feature_vector'])

In [36]:
all_cluster_assignments = data.get_cluster_assignments(64, projected_45, projected_45[index])

In [37]:
len(all_cluster_assignments)

3388858

In [38]:
len(d['feature_vector'])

3388858

Converting cluster assignments to truck/no truck

In [39]:
truck_clusters = [5, 10, 11, 18, 20, 37, 42, 57, 63]

In [46]:
def truck_label(x):
    if x in truck_clusters:
        return 1
    else:
        return 2

In [47]:
all_truck_labels = [truck_label(x) for x in all_cluster_assignments]

In [49]:
truck_label_df = pd.DataFrame(data={'timestamp': d['timestamp'], 'sensor_id': d['sensor_id'], \
                                   'filepath': d['file_path'], 'label': all_truck_labels})

In [50]:
truck_label_df.head()

,timestamp,sensor_id,filepath,label
0,1.559362e+09,b'sonycnode-b827ebc178d2.sonyc',b'/Users/marin/redhook/embeddings/sonycnode-b8...,2
1,1.559362e+09,b'sonycnode-b827ebc178d2.sonyc',b'/Users/marin/redhook/embeddings/sonycnode-b8...,2
2,1.559362e+09,b'sonycnode-b827ebc178d2.sonyc',b'/Users/marin/redhook/embeddings/sonycnode-b8...,2
3,1.559362e+09,b'sonycnode-b827ebc178d2.sonyc',b'/Users/marin/redhook/embeddings/sonycnode-b8...,2
4,1.559362e+09,b'sonycnode-b827ebc178d2.sonyc',b'/Users/marin/redhook/embeddings/sonycnode-b8...,2
